# 통합 심층 분석 파이프라인

1. 프로젝트 개요

'카카오톡 업데이트 사용자 피백 분석' 프로젝트의 마지막 단계인 심층 분석(Deep-dive Analysis)을 수행하는 통합 파이프라인입니다.
이전 단계에서 수집 및 정제된 각 데이터 소스(구글 리뷰, 뉴스, 블로그)를 일관된 기준과 방법론을 적용하여 다각적인 비교 분석을 수행하는 것을 목표로 합니다.

2. 설계 방향: 설정과 실행의 분리

심층 분석 코드는 <b>재사용성과 확장성</b>을 극대화하기 위해 아래의 두 가지 핵심 원칙에 따라 설계되었습니다.

- 모듈화된 분석 함수: 키워드 추출(Step 1)부터 네트워크 분석(Step 5)까지, 각 분석 단계를 <b>독립적인 함수(Function)</b>로 정의했습니다. 이를 통해 특정 분석만 독립적으로 실행하거나, 새로운 분석 기법을 쉽게 추가할 수 있습니다.

- 설정 기반 파이프라인: 하단의 '분석 환경 설정' 영역에서 ANALYSIS_TARGET, SOURCE_DATA_PATH 등 몇 가지 파라미터만 변경하면, 정의된 함수들을 그대로 사용하여 어떤 종류의 데이터(구글 리뷰, 뉴스, 블로그)든, 어떤 분석 구간이든 동일한 품질의 심층 분석을 자동으로 수행할 수 있습니다.

3. 주요 분석 단계
- Step 1. 키워드 추출: Okt 형태소 분석기를 사용하여 각 문서의 핵심 명사 키워드를 추출하고, 이후 분석을 위해 .pkl 파일로 중간 저장합니다.
- Step 2. 키워드 빈도 분석: 각 감성 그룹별 상위 키워드를 시각화하여 가장 많이 언급된 주제를 파악합니다.
- Step 3. N-gram 분석: 단어들의 연결 관계를 분석하여, 사용자들이 실제로 사용한 구체적인 '구문'과 '맥락'을 파악합니다.
- Step 4. LDA 토픽 모델링: 문서 군집에 숨어있는 핵심 '주제(Topic)'들을 자동으로 추출하여, 여론의 주요 쟁점을 구조적으로 파악합니다.
- Step 5. 네트워크 분석: 키워드 간의 동시 출현 빈도를 시각화하여, 어떤 키워드가 여론의 '중심'에 있고, 각 키워드들이 어떻게 서로 연결되어 있는지를 분석합니다. *(블로그는 Step 5 적용시 메모리 초과 문제가 발생되어 분할하여 작업을 진행할 수 있는 수정 코드를 적용하였습니다.)*

In [ ]:
# 그래프 한글 나눔 폰트 설치 및 설정
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# 런타임 > 세션 다시 시작 실행

In [ ]:
# 형태소 분석기 및 워드 클라우드 라이브러리 설치
!pip install konlpy wordcloud gensim pyLDAvis networkx tqdm nltk

# NLTK 데이터 다운로드 (punkt)
import nltk
nltk.download('punkt')

In [ ]:
# ==============================================================================
# setting
# ==============================================================================

# 라이브러리 임포트
import pandas as pd
import numpy as np
import os
import re
from tqdm.auto import tqdm; tqdm.pandas()
import warnings

# 텍스트 분석
from konlpy.tag import Okt
from collections import Counter
from itertools import combinations
from nltk import ngrams
import gensim
from gensim import corpora

# 시각화
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns
from wordcloud import WordCloud
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# 경고 메시지때문에 결과 정보 확인이 어려워서 숨기기 기능 추가
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
# ==============================================================================
# 1 - 5 단계별 분석/시각화 함수 정의(구글, 뉴스, 블로그 공통)
# ==============================================================================

# help : 한글 폰트 확인 및 설정 (폰트 없으면 중단)
def check_and_set_korean_font(stop_on_fail=True):
    # 시각화를 위한 한글 폰트 확인 후 폰트가 없을 경우 분석 중단

    font_name = 'NanumBarunGothic'

    if not any(font.name == font_name for font in fm.fontManager.ttflist):
        print(f"[경고] '{font_name}' 폰트가 설치되어 있지 않습니다. 그래프의 한글이 깨질 수 있습니다.")
        print("-> 코랩(Colab) 환경이라면, 상단 메뉴 '런타임' -> '런타임 다시 시작' 후 아래 코드를 실행하여 폰트를 설치하세요.")
        print("!sudo apt-get -y install fonts-nanum && sudo fc-cache -fv && rm ~/.cache/matplotlib -rf")

        if stop_on_fail:
            raise SystemExit("\n✋ 폰트가 설치되지 않아 분석을 중단합니다. 폰트 설치 후 다시 시도해주세요.")

    else:
        print(f"✅ '{font_name}' 폰트가 준비되었습니다. 시각화를 진행합니다.")

    # 폰트가 성공적으로 확인된 경우 설정 실행
    plt.rc('font', family=font_name)
    plt.rcParams['axes.unicode_minus'] = False

# help : 뉴스/블로그에서 제목과 본문을 나눠서 분석해야 해서 처리
def reshape_news_blog_data(df, title_col, title_sentiment_col, body_col, body_sentiment_col):
    # 뉴스/블로그 데이터를 분석하기 쉽게 Long Format으로 변환 후 별도 행으로 분리

    print("뉴스/블로그 데이터를 분석용 구조로 변환합니다...")

    # 1. 제목 데이터프레임 생성
    df_title = df[[title_col, title_sentiment_col]].copy()
    df_title.columns = ['text', 'sentiment']
    df_title['source_type'] = '제목'

    # 2. 본문 데이터프레임 생성
    df_body = df[[body_col, body_sentiment_col]].copy()
    df_body.columns = ['text', 'sentiment']
    df_body['source_type'] = '본문'

    # 3. 두 데이터프레임을 위아래로 합치기
    df_reshaped = pd.concat([df_title, df_body], ignore_index=True)

    # 4. 분석 그룹을 명확히 하기 위해 새로운 컬럼 생성 (예: '제목_긍정', '본문_부정')
    df_reshaped['analysis_group'] = df_reshaped['source_type'] + '_' + df_reshaped['sentiment']

    # 5. 비어있는 텍스트 행은 분석에서 제외
    df_reshaped = df_reshaped.dropna(subset=['text'])
    df_reshaped = df_reshaped[df_reshaped['text'].str.strip() != '']

    print("✅ 데이터 구조 변환 완료!")
    return df_reshaped

# [1단계] 키워드 추출 및 저장 함수
def step1_extract_and_save_keywords(df, text_columns, stopwords, output_path):
    print(f"\n[STEP 1] 키워드 추출을 시작합니다... (시간이 다소 소요될 수 있습니다)")
    okt = Okt()
    processed_df = df.copy()
    processed_df['통합본문'] = processed_df[text_columns].fillna('').agg(' '.join, axis=1)

    def extract_keywords(text):
        text = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣\s]', '', str(text))
        nouns = okt.nouns(text)
        return [word for word in nouns if len(word) > 1 and word not in stopwords]

    processed_df['keywords'] = processed_df['통합본문'].progress_apply(extract_keywords)
    processed_df.to_pickle(output_path)
    print(f"✅ 키워드 추출 완료! 결과가 '{output_path}' 파일로 저장되었습니다.")
    return processed_df

# [2단계] 빈도 분석 (데이터 계산)
def step2_calculate_frequency(df_processed, sentiment_column):
    print("\n[STEP 2] 키워드 빈도 분석 데이터 계산 중...")
    results = {
        sentiment: Counter(sum(df_processed[df_processed[sentiment_column] == sentiment]['keywords'], []))
        for sentiment in sorted(df_processed[sentiment_column].unique())
    }
    print("✅ 빈도 데이터 계산 완료!")
    return results

def visualize_frequency(sentiment, word_counts, top_n=20, save_prefix=""):
    print(f"  -> '{sentiment}' 그룹 빈도 분석 시각화...")
    most_common_words = word_counts.most_common(top_n)
    df_words = pd.DataFrame(most_common_words, columns=['word', 'count'])

    plt.figure(figsize=(12, 8))
    sns.barplot(x='count', y='word', data=df_words, palette='viridis')
    plt.title(f'"{save_prefix}{sentiment}" 상위 {top_n}개 키워드', fontsize=16)
    plt.show()

    wordcloud = WordCloud(font_path='NanumBarunGothic', width=800, height=500, background_color='white').generate_from_frequencies(dict(word_counts))
    plt.figure(figsize=(10, 7))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(f'"{save_prefix}{sentiment}" 워드 클라우드', fontsize=16)
    plt.axis('off')
    plt.show()

# [3단계] N-gram 분석 (데이터 계산 및 시각화)
def step3_analyze_and_visualize_ngrams(df_processed, sentiment_column, n=2, top_n=20, save_prefix=""):
    print(f"\n[STEP 3] {n}-gram 분석 및 시각화를 시작합니다...")
    for sentiment in sorted(df_processed[sentiment_column].unique()):
        all_words = sum(df_processed[df_processed[sentiment_column] == sentiment]['keywords'], [])
        if len(all_words) < n: continue

        ngrams_list = list(ngrams(all_words, n))
        ngram_counts = Counter(ngrams_list)
        most_common_ngrams = [(' '.join(gram), count) for gram, count in ngram_counts.most_common(top_n)]

        df_ngrams = pd.DataFrame(most_common_ngrams, columns=[f'{n}-gram', 'count'])

        print(f"  -> '{sentiment}' 그룹 {n}-gram 시각화...")
        plt.figure(figsize=(12, 8))
        sns.barplot(x='count', y=f'{n}-gram', data=df_ngrams, palette='plasma')
        plt.title(f'"{save_prefix}{sentiment}" 상위 {top_n}개 {n}-gram', fontsize=16)
        plt.show()

# [4단계] LDA 토픽 모델링 (학습 및 HTML 저장)
def step4_run_lda_modeling(df_processed, sentiment_column, num_topics=4, save_prefix=""):
    print("\n[STEP 4] LDA 토픽 모델링을 시작합니다...")
    for sentiment in sorted(df_processed[sentiment_column].unique()):
        print(f"  -> '{sentiment}' 그룹 LDA 모델 학습 및 저장...")
        documents = df_processed[df_processed[sentiment_column] == sentiment]['keywords'].tolist()
        documents = [doc for doc in documents if doc] # 비어있는 리스트 제거
        if not documents: continue

        dictionary = corpora.Dictionary(documents)
        corpus = [dictionary.doc2bow(text) for text in documents]

        ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15, random_state=42)

        vis_data = gensimvis.prepare(ldamodel, corpus, dictionary, mds='tsne')
        html_file = f'{save_prefix}{sentiment}_LDA_결과.html'
        pyLDAvis.save_html(vis_data, html_file)
        print(f"    ✅ LDA 시각화 파일 저장 완료: {html_file}")

# [5단계] 네트워크 분석 (데이터 계산 및 시각화)
def step5_analyze_and_visualize_network(df_processed, sentiment_column, word_freq_results, top_n=50, save_prefix=""):
    print("\n[STEP 5] 네트워크 분석 및 시각화를 시작합니다...")
    for sentiment in sorted(df_processed[sentiment_column].unique()):
        documents = df_processed[df_processed[sentiment_column] == sentiment]['keywords'].tolist()
        documents = [doc for doc in documents if doc]
        if not documents: continue

        pairs = [combination for keywords in documents if len(keywords) >= 2 for combination in combinations(set(keywords), 2)]
        if not pairs: continue

        pair_counts = Counter(pairs)
        df_edges = pd.DataFrame(pair_counts.most_common(top_n), columns=['pair', 'count'])
        df_edges[['source', 'target']] = pd.DataFrame(df_edges['pair'].tolist(), index=df_edges.index)
        G = nx.from_pandas_edgelist(df_edges, source='source', target='target', edge_attr='count')

        print(f"  -> '{sentiment}' 그룹 네트워크 시각화...")
        plt.figure(figsize=(18, 18))
        pos = nx.spring_layout(G, k=0.8, iterations=100, seed=42)
        node_sizes = [np.sqrt(word_freq_results[sentiment].get(node, 1)) * 300 for node in G.nodes()]
        edge_widths = [d['count'] * 0.05 for (u, v, d) in G.edges(data=True)]

        nx.draw_networkx(G, pos,
                         font_family='NanumBarunGothic', font_size=14,
                         node_size=node_sizes, node_color=list(dict(G.degree()).values()),
                         cmap=plt.cm.viridis, alpha=0.8,
                         width=edge_widths, edge_color='grey')

        plt.title(f'"{save_prefix}{sentiment}" 키워드 네트워크 분석', fontsize=25)
        plt.axis('off')
        plt.show()

In [ ]:
# [5단계] 네트워크 분석 (메모리 최적화 버전) 블로그 분석 시 사용  >> 블로그 네트워크 분석 안되서. 추가

import gc # Garbage Collector 라이브러리 추가

def step5_analyze_and_visualize_network_optimized(df_processed, sentiment_column, word_freq_results, top_n=40, save_prefix="", chunk_size=10000):
    print("\n[STEP 5] 네트워크 분석 및 시각화 (메모리 최적화 버전)을 시작합니다...")
    for sentiment in sorted(df_processed[sentiment_column].unique()):
        print(f"  -> '{sentiment}' 그룹 네트워크 데이터 계산 중...")
        documents = df_processed[df_processed[sentiment_column] == sentiment]['keywords'].tolist()
        documents = [doc for doc in documents if doc]
        if not documents: continue

        # 🔥 전체 pairs를 한 번에 만들지 않고, Counter를 점진적으로 업데이트
        pair_counts = Counter()
        for i in tqdm(range(0, len(documents), chunk_size), desc="  - 단어 쌍 계산 진행률"):
            chunk_documents = documents[i:i+chunk_size]
            pairs_chunk = [
                combination for keywords in chunk_documents
                if len(keywords) >= 2
                for combination in combinations(set(keywords), 2)
            ]
            pair_counts.update(pairs_chunk)
            del pairs_chunk # 메모리에서 즉시 해제
            gc.collect()  # 가비지 컬렉션 강제 실행

        if not pair_counts: continue

        df_edges = pd.DataFrame(pair_counts.most_common(top_n), columns=['pair', 'count'])
        df_edges[['source', 'target']] = pd.DataFrame(df_edges['pair'].tolist(), index=df_edges.index)
        G = nx.from_pandas_edgelist(df_edges, source='source', target='target', edge_attr='count')

        print(f"  -> '{sentiment}' 그룹 네트워크 시각화...")
        plt.figure(figsize=(18, 18))
        pos = nx.spring_layout(G, k=0.8, iterations=100, seed=42)
        node_sizes = [np.sqrt(word_freq_results[sentiment].get(node, 1)) * 300 for node in G.nodes()]
        edge_widths = [d['count'] * 0.05 for (u, v, d) in G.edges(data=True)]

        nx.draw_networkx(G, pos,
                         font_family='NanumBarunGothic', font_size=14,
                         node_size=node_sizes, node_color=list(dict(G.degree()).values()),
                         cmap=plt.cm.viridis, alpha=0.8,
                         width=edge_widths, edge_color='grey')

        plt.title(f'"{save_prefix}{sentiment}" 키워드 네트워크 분석', fontsize=25)
        plt.axis('off')
        plt.show()

In [ ]:
# ==============================================================================
# [최종] 실 적용 환경 설정 및 함수 실행 부분 >> 구글, 뉴스, 블로그 세팅 진행
# ==============================================================================

# 1. 분석 환경 설정

# [ 구글 스토어 리뷰 ]
ANALYSIS_TARGET = "google_stroe"
SOURCE_DATA_PATH = '/content/drive/MyDrive/Colab Notebooks/251118_구글스토어_감성분석결과.csv' # 원본 데이터 파일 경로
PROCESSED_DATA_PATH = '/content/drive/MyDrive/Colab Notebooks/251201_구글리뷰_keywords.pkl'      # 중간 저장될 파일 경로
SAVE_PREFIX = '구글리뷰_'                         # 결과물 파일 이름 접두사
TEXT_COLUMNS = ['cleaned_content']              # 분석할 텍스트 컬럼 (2개 이상 가능: ['제목', '본문'])
SENTIMENT_COLUMN = 'sentiment_integrated'       # 감성 분석 결과 컬럼

# [ 네이버 뉴스-news, 블로그-blog ]
# ANALYSIS_TARGET = 'blog'
# SOURCE_DATA_PATH = '블로그_데이터_감성분석결과.csv'
# PROCESSED_DATA_PATH = '블로그_keywords.pkl'
# SAVE_PREFIX = '블로그_'
# # 아래 컬럼 이름들은 실제 데이터에 맞게 수정 필요
# TITLE_COL = '제목'
# TITLE_SENTIMENT_COL = '제목_감성'
# BODY_COL = '본문'
# BODY_SENTIMENT_COL = '본문_감성'

# 분석에 사용될 공통 불용어 리스트
common_stopwords = [
    '등', '수', '이', '것', '저', '그', '때', '및', '의', '를', '에', '가', '들', '좀', '잘', '더', '그냥', '과',
    '카카오', '카톡', '업데이트', '업뎃' , '이번', '버전', '기자', '뉴스', '사진', '제공', '무단', '전재'
]

# 2. 분석 파이프라인 실행

print(f"--- '{SAVE_PREFIX}' 전체 분석을 시작합니다 ---")
try:
    check_and_set_korean_font(stop_on_fail=True)
except SystemExit as e:
    # 폰트 문제로 중단되었을 경우 메시지를 출력 후 종료
    print(e)
else:
    # 폰트가 정상적일 경우에만 아래의 모든 분석 실행

    # [STEP 1] 키워드 추출
    if not os.path.exists(PROCESSED_DATA_PATH):
        print(f"\n[STEP 1] '{PROCESSED_DATA_PATH}' 파일이 없어 키워드 추출을 시작합니다.")
        df_raw = pd.read_csv(SOURCE_DATA_PATH)
            # 분석 대상에 따라 데이터 구조 결정
        if ANALYSIS_TARGET in ['blog', 'news']:
            df_to_process = reshape_news_blog_data(df_raw, TITLE_COL, TITLE_SENTIMENT_COL, BODY_COL, BODY_SENTIMENT_COL)
            # 키워드 추출 함수에는 변환된 컬럼 이름을 전달
            df_processed = step1_extract_and_save_keywords(df_to_process, ['text'], common_stopwords, PROCESSED_DATA_PATH)
            # 분석 그룹(sentiment) 컬럼을 새로 만든 'analysis_group'으로 지정
            SENTIMENT_COLUMN = 'analysis_group'
        else: # 구글 리뷰 또는 기타 단일 텍스트 데이터
            df_processed = step1_extract_and_save_keywords(df_raw, TEXT_COLUMNS, common_stopwords, PROCESSED_DATA_PATH)
    else:
        print(f"\n[SKIP] 이미 '{PROCESSED_DATA_PATH}' 파일이 존재하여 1단계를 건너뛰고 파일을 불러옵니다.")
        df_processed = pd.read_pickle(PROCESSED_DATA_PATH)
        # 저장된 파일을 불러올 때도 분석 대상에 따라 SENTIMENT_COLUMN을 재정의 해줘야 함
        if ANALYSIS_TARGET in ['blog', 'news']:
            SENTIMENT_COLUMN = 'analysis_group'

# [STEP 2] 키워드 빈도 분석 및 시각화
word_freq_results = step2_calculate_frequency(df_processed, SENTIMENT_COLUMN)
for sentiment, counts in word_freq_results.items():
    if counts:
        visualize_frequency(sentiment, counts, save_prefix=SAVE_PREFIX)

# # [STEP 3] N-gram 분석 및 시각화
step3_analyze_and_visualize_ngrams(df_processed, SENTIMENT_COLUMN, save_prefix=SAVE_PREFIX)

# [STEP 4] LDA 토픽 모델링
step4_run_lda_modeling(df_processed, SENTIMENT_COLUMN, num_topics=4, save_prefix=SAVE_PREFIX)

# [STEP 5] 네트워크 분석 및 시각화
# 2단계에서 계산한 단어 빈도 결과를 노드 크기 조절에 사용
step5_analyze_and_visualize_network(df_processed, SENTIMENT_COLUMN, word_freq_results, save_prefix=SAVE_PREFIX)

print("\n 모든 분석이 성공적으로 완료되었습니다. ")